In [1]:
import fitparse as Fit

In [2]:
ride = Fit.FitFile('Rides/NCAR.fit')

In [3]:
seen = {}
for x in ride:
    if str(x) not in seen:
        print(x)
        seen[str(x)] = True

file_id (#0)
developer_data_id (#207)
field_description (#206)
event (#21)
device_info (#23)
unknown_65281 (#65281)
sport (#12)
workout (#26)
record (#20)
hr_zone (#8)
power_zone (#9)
unknown_65280 (#65280)
lap (#19)
segment_lap (#142)
unknown_65282 (#65282)
session (#18)
activity (#34)


In [4]:
i = 0
for record in ride.get_messages('lap'):
    for data in record:
        print(data.name, data.value)
    if i > 100:
        break
    i += 1
    print()

avg_altitude 1630.6
avg_cadence 76
avg_grade 1.17
avg_heart_rate 153
avg_power 135
avg_speed 5.392
avg_temperature 33
enhanced_avg_altitude 1630.6
enhanced_avg_speed 5.392
enhanced_max_altitude 1692.4
enhanced_max_speed 9.496
enhanced_min_altitude 1586.1999999999998
event lap
event_type stop
max_altitude 1692.4
max_cadence 103
max_heart_rate 180
max_neg_grade -3.81
max_pos_grade 7.71
max_power 537
max_speed 9.496
max_temperature 37
min_altitude 1586.1999999999998
min_heart_rate 123
normalized_power 160
start_time 2020-08-07 17:20:26
time_in_hr_zone (64.436, 486.151, 157.104, 186.209, 10.304)
time_in_power_zone (283.686, 395.381, 164.099, 14.894, 11.489, 32.798)
timestamp 2020-08-07 17:35:36
total_ascent 114
total_calories 123
total_descent 12
total_distance 4890.48
total_elapsed_time 910.0
total_timer_time 907.0
total_work 122968

avg_altitude 1761.6
avg_cadence 98
avg_grade 3.5
avg_heart_rate 187
avg_power 297
avg_speed 5.566
avg_temperature 39
enhanced_avg_altitude 1761.6
enhanced_av